In [8]:
# #Flask
# !pip3 install flask
# !pip3 install pyperclip
# !pip3 install selenium
# !pip3 install Cx_Oracle
# !pip3 install BeautifulSoup4
# !pip3 install werkzeug
# !pip3 install flask_cors
# !pip3 install requests
# # MTCNN
# !pip3 install facenet
# !pip3 install MTCNN
# !pip3 install facenet_pytorch
# !pip3 install opencv-python
# !pip3 install MMCV
# !pip3 install IPython
# !pip3 install Ipython display
# # Dlib 
# !pip3 install cmake
# !pip3 install dlib 
# !pip3 install imutils
# !pip3 install scipy
#!pip3 install PyMongo

In [103]:
import requests as req
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, request, Response, redirect, url_for, abort, stream_with_context, send_file, jsonify
import os, sys, time, pyperclip, io
from werkzeug.utils import secure_filename
from wsgiref.simple_server import make_server
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from flask_cors import CORS, cross_origin
import base64, json, shlex
# MTCNN 
from facenet_pytorch import MTCNN 
import mmcv, cv2, torch
from PIL import Image, ImageDraw
from torchvision import datasets, transforms
import joblib
#HTML Video
import subprocess as sp
import threading
from IPython.display import HTML, display
from base64 import b64encode
import binascii
import codecs
# Dlib
import imutils
from imutils import face_utils
# Euclidean distance
from scipy.spatial import distance
from math import sqrt
import pandas as pd
import dlib
from werkzeug.datastructures import FileStorage # 대용량 파일 다루기
from tempfile import TemporaryFile
# Thread
from threading import Thread
from multiprocessing import Process, Queue
# DB
import cx_Oracle

In [134]:
app=Flask(__name__) # 플라스크 서버 객체 생성

CORS(app)

CORS(app, resources={r'*': {'origins': '*'}})

# 사용자 요청을 처리하기 위한 라우터 설정
@app.route('/render', methods=["GET",'POST'])
def render_file():
    return render_template("upload.html") # 업로드용 페이지 html 보여주기

#파일 업로드 처리
@app.route("/fileUpload", methods=['GET',"POST"])
def file_upload():
    
    if request.method=='POST':
        f = request.files["file"] # html <input name= "file">
        global FILE_NAME
        
        FILE_NAME = f.filename 
        f.save(secure_filename(f.filename)) # test_park.mp4
        print(1)
        img= MTCNN_model(os.path.join("./", FILE_NAME)) # MTCNN
        for i,face in enumerate (img):  # 검출된 얼굴을 하드에 저장하기 위한 for문
            img_path = face.save(f"./static/{FILE_NAME}.jpg") 
            img_path = face.save(f"./static/{FILE_NAME}{i}.jpg") 
        print(2)
        print(f"img : {img} len : {len(img)} type : {type(img)}")
        d_list = list() # 검출된 얼굴 유클 거리 계산이 끝난 결과가 저장된 리스트
        
        land = DLIB_model(img) # 랜드마크 검출
        
        if land: 
            for i2 in land:
                d_list.append(euclidean_distance_calculator(i2)) 
            print(3)
            for i3 in d_list: # DB 접근
                insert_face(i3)    
            print(4)
            
#             insert_DB = Thread(target=insert_face, args=(d_list)) # DB 입력 쓰레드
#             insert_DB.start()
#             insert_DB.join()

        return redirect("http://59.0.236.2:2000/view2")
    
    
# @app.route('/sendFrame',methods=["GET","POST"])
# def sendFrame():
#     byteData = request.get_data()
#     try:
#         frame = Image.frombuffer('RGBA', (320,240), byteData)
#         frame = frame.rotate(180)
#         display(frame)
#     except:
#         print("No Exist Buffer")
        
#     return 'Hello World!'
    
@app.route("/view") 
def view(): 
    return render_template("view.html", image_file=f"./static/{FILE_NAME}.jpg") # html페이지에 담아서 보내기

@app.route("/view2")
def view2():
    return send_file(f"./static/{FILE_NAME}.jpg") # 이미지 파일만 보내기


# 서버 구동
if __name__ == "__main__":
    app.run(host='59.0.236.2',port=2000)
    app.debug=True
#     59.0.236.2

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://59.0.236.2:2000/ (Press CTRL+C to quit)


In [ ]:
app = Flask(__name__)
  
def gen_frames():
#     cam = find_camera(camera_id)  # return the camera access link with credentials. Assume 0?
    # cam = cameras[int(id)]
    cap = cv2.VideoCapture(0)  # capture the video from the live feed
 
    while True:
 
        # # Capture frame-by-frame. Return boolean(True=frame read correctly. )
        success, frame = cap.read()  # read the camera frame
        if not success:
            br

# DB 연동

In [132]:
def insert_face(distance_list):
    
    LOCATION = r"C:\instantclient_21_3"
    os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"] #환경변수
    
    user = "suhyun"
    pw = "rhdwn123"
    url = "project-db-stu.ddns.net:1524"
    data = ""
    
    connect = cx_Oracle.connect(user, pw, url) # DB연결
    cursor=connect.cursor()
    try:
        sql = "INSERT INTO T_FACE (FACE_SEQ, EUCLIDEAN_DISTANCE) VALUES (t_face_SEQ.NEXTVAL,:1)"
        for count, dist in enumerate(distance_list):
            
            if count != (len(distance_list)-1):
                data += str(dist)+" "
#                 data = str(data)
            else:
                data += str(dist)
        
        print(f'data >>>> {data} dtype : {type(data)}')
        print(len(data))
        #cursor.execute(sql,data) # 전송
        #sql = "select * from t_face"
        datas = [data]
        cursor.execute(sql,datas)
        
#         for i in cursor:
#             print(f"cursor : {i}")

    except Exception as e:
        print(f"DB insert_error {e}")
    finally:
        cursor.close()
        connect.commit()
        connect.close()
        print("insert finish")
    return ""

# 모델 선언 (실시간)

# 모델 선언 (파일 업로드 버전)

In [21]:
# MTCNN 
def MTCNN_model(v_path):
    cutted_face_location = []
    frame_list = []
    detected_rectangle_faces = []

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  
    mtcnn = MTCNN(keep_all=True, device=device) 
    video = mmcv.VideoReader(v_path)
    frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]
    for i, frame in enumerate(frames):  
        boxes, _ = mtcnn.detect(frame)
        frame_draw = frame.copy()
        draw = ImageDraw.Draw(frame_draw)
        try:
            for box in boxes:
                if box is not None :
                    draw.rectangle(box.tolist(), outline=(0, 0, 0), width=1) 
                    frame_list.append(frame_draw)
                    cutted_face_location.append(box)
        except:
            print("landmark point error")
    for i in range(len(frame_list)):
        x,y,w,h=cutted_face_location[i]
        croppedImage=frame_list[i].crop((x,y,w,h))
        detected_rectangle_faces.append(croppedImage)
        display(croppedImage)

    cutted_face_location.clear()
    frame_list.clear()
    
    return detected_rectangle_faces

    
def DLIB_model(detected_rectangle_faces):

    nparray_detected_rectangle_faces = []
    coor = []
    try:
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")#D
    except:
        print("Dlib Predictor read error")
    
    for i1 in detected_rectangle_faces:
        nparray_detected_rectangle_faces.append(np.array(i1))
        for j in nparray_detected_rectangle_faces:
            image = cv2.cvtColor(j, cv2.COLOR_RGB2BGR)
            imgae = cv2.resize(image,(480,480),interpolation = cv2.INTER_CUBIC)
            image = imutils.resize(image, width=480)
            color_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            rectangle = detector(color_image, 1)

            for(i, rect) in enumerate(rectangle):  
                LAND = predictor(color_image, rect) #D
                LAND = face_utils.shape_to_np(LAND)
                coor.append(LAND)
 
                (x,y,w,h) = face_utils.rect_to_bb(rect)
                cv2.rectangle(color_image, (x,y), (x+w, y+h), (0,255,255), 1)
                cv2.putText(color_image, "Face #{}".format(i+1), (x-1, y-1), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255,0,255), 1)
                
    return coor

def euclidean_distance_calculator(vector_dim): 
    dlist = []
    for i in range(68):
        if i != 30:
            dlist.append(distance.euclidean(vector_dim[30],vector_dim[i])) # 유클리디언 거리
    return dlist

In [7]:
def WebCam(v_path):
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # cuda GPU 를 사용하며, 없을 시 cpu 사용
    print('Running on device: {}'.format(device))

    mtcnn = MTCNN(keep_all=True, device=device) # mtcnn 모델 가져오기

    video = mmcv.VideoReader(v_path) # mmcv 비디오 읽어오기
    frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video] # 프레임별로 cvt 시킨 후 저장


    storage_face = [] # 탐지가 완료된 이미지 자체를 저장한 리스트
    cutted_face_location = [] # 얼굴 부분만 잘라낸 좌표들을 저장하는 리스트
    frame_list = []
    # enumerate : 반복문 사용 시 몇 번째 반복문인지 확인할 떄 사용, 인덱스 번호와 원소를 tuple형태로 반환합니다.
    for i, frame in enumerate(frames): # frame : 튜플형태로 사진 데이터 저장 / i : 인덱스 번호
        
        print('\r프레임: {}'.format(i + 1)) # 각각의 프레임 출력
        # 얼굴 탐지
        boxes, _ = mtcnn.detect(frame) # mtcnn 모델을 활용하여 한 프레임씩 탐지 후 박스에 결과 저장 
        # boxes : 2차원의 배열 (21, 4) # 행 : 감지된 사람의 얼굴 수 / 열 : 해당 정보의 차원 및 좌표
        # _ :  프레임 한 장에 탐지된 사람들의 얼굴 확률 값

        # 얼굴 그리기
        frame_draw = frame.copy() # 프레임 한 장 카피
        draw = ImageDraw.Draw(frame_draw) 
        
    try: 
      # 감지된 사람 수가 0이라면 모델에 오류가 발생함 이를 해결하기 위한 알고리즘
      # 얼굴만 탐지하는 BOX
        for box in boxes:
            if box is not None :

                # 네모 그리기
                draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6) # tolist : list 형태로 대상의 차원에 맞춰서 반환, 얼굴 탐지가 된 좌표값
                #outline : 네모 색상 / width : 두께

                frame_list.append(frame_draw)
                cutted_face_location.append(box) # 검출된 얼굴 좌표


                # storage_face.append(frame_draw.resize((224, 224), Image.BILINEAR)) # 사이즈 축소 및 임시 저장소 리스트에 추가
                # BILINEAR : 이미지 리사이즈 시 이진선형을 사용하여 리사이즈 하겠다.
    except:
         print("except!!") # 가독성을 위한 띄어쓰기
    return 

# DB연결 